# Introducción a APIs de Modelos de Lenguaje

## Objetivos de Aprendizaje

En este cuaderno vas a aprender:
- Qué es una API y cómo nos permite acceder a modelos de lenguaje grandes (LLMs)
- Cómo configurar y autenticar tu conexión con OpenAI
- Técnicas de prompting básicas para extraer información estructurada
- Casos de uso prácticos del procesamiento del lenguaje natural

## ¿Qué es una API?

Una **API** (Application Programming Interface) es una interfaz que permite que dos sistemas se comuniquen entre sí. En nuestro caso, nos permite enviar texto a un modelo de lenguaje (como GPT-4) y recibir una respuesta procesada.

**Analogía**: Imaginate que estás en un restaurante. Vos sos el cliente (tu programa), el mozo es la API, y la cocina es el modelo de lenguaje. Vos le pedís algo al mozo (enviás un prompt), el mozo se lo lleva a la cocina (el modelo procesa), y te trae de vuelta el plato (la respuesta).

## Prerequisitos

Para usar este cuaderno necesitás:
1. Una cuenta en OpenAI (https://platform.openai.com)
2. Una API key (se genera en https://platform.openai.com/account/api-keys)
3. Crédito en tu cuenta de OpenAI (hay planes gratuitos para empezar)

**Nota sobre costos**: Los modelos como `gpt-4o-mini` son muy económicos (menos de $0.01 por cada 1000 palabras procesadas). Para este curso, con $5 USD tenés suficiente para todos los ejercicios.

## Configuración del Entorno

Primero instalamos la librería de OpenAI y configuramos la autenticación. Este código detecta automáticamente si estás en Google Colab o trabajando localmente, y carga la API key desde el lugar apropiado.

In [ ]:
# Instalamos la librería oficial de OpenAI
!pip install --upgrade openai --quiet

import os
from openai import OpenAI

# Detectamos si estamos en Google Colab o en entorno local
IN_COLAB = False
try:
    import google.colab
    from google.colab import userdata
    IN_COLAB = True
except Exception:
    IN_COLAB = False

# Cargamos la API key según el entorno
OPENAI_API_KEY = None
if IN_COLAB:
    # En Colab: usar secrets (icono de llave en la barra lateral)
    try:
        OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    except Exception:
        OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
else:
    # En local: usar variable de entorno o archivo .env
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

if not OPENAI_API_KEY:
    raise ValueError(
        'OPENAI_API_KEY no encontrada.\n'
        'En Colab: agregala en Secrets (icono de llave)\n'
        'En local: agregala a tus variables de entorno o archivo .env'
    )

print(f"API key cargada correctamente. Entorno: {'Colab' if IN_COLAB else 'Local'}")

### Notas Técnicas: Configuración de API Key

**En Google Colab**:
1. Clic en el icono de llave en la barra lateral izquierda
2. Agregar un nuevo secret con nombre `OPENAI_API_KEY`
3. Pegar tu API key como valor

**En entorno local**:
```bash
# Opción 1: Variable de entorno (Linux/Mac)
export OPENAI_API_KEY='tu-api-key-aqui'

# Opción 2: Archivo .env
# Crear archivo .env en la raíz del proyecto:
OPENAI_API_KEY=tu-api-key-aqui
```

**Seguridad**: Nunca subas tu API key a GitHub o la compartas públicamente. Las API keys son como contraseñas y dan acceso a tu cuenta (y crédito) de OpenAI.

## Función Auxiliar: get_completion()

Creamos una función helper que simplifica el envío de prompts a la API. Esta función encapsula toda la lógica de comunicación y nos permite concentrarnos en el contenido de nuestros prompts.

### Parámetros configurables:

- **prompt** (str): El texto que queremos enviar al modelo
- **model** (str): Qué modelo usar. Opciones comunes:
  - `gpt-4o-mini`: Rápido, económico, ideal para la mayoría de tareas
  - `gpt-4o`: Más potente, mejor razonamiento, más caro
  - `gpt-4-turbo`: Balance entre velocidad y capacidad
- **temperature** (float, 0-2): Controla la aleatoriedad de las respuestas
  - `0.0`: Determinístico, siempre responde igual (ideal para extracción de datos)
  - `0.7`: Balance (bueno para texto general)
  - `1.5-2.0`: Creativo, más variado (útil para generación creativa)

**Recomendación**: Para tareas de extracción y análisis, usá temperature=0. Para generación creativa, probá con valores entre 0.7 y 1.0.

In [ ]:
def get_completion(prompt, model="gpt-4o-mini", temperature=0):
    """
    Envía un prompt a la API de OpenAI y devuelve la respuesta.
    
    Parámetros:
    -----------
    prompt : str
        El texto que queremos enviar al modelo
    model : str, default="gpt-4o-mini"
        Modelo a utilizar (gpt-4o-mini es económico y efectivo)
    temperature : float, default=0
        Grado de aleatoriedad (0 = determinístico, 2 = muy creativo)
    
    Returns:
    --------
    str
        La respuesta generada por el modelo
    """
    # Creamos el cliente de OpenAI
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    # Estructuramos el mensaje como conversación
    messages = [{"role": "user", "content": prompt}]
    
    # Hacemos el request a la API
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    
    # Extraemos y devolvemos solo el contenido de la respuesta
    return response.choices[0].message.content

## Ejemplo 1: Análisis de Sentimiento en Reviews

Vamos a analizar una review de MercadoLibre y extraer información estructurada. Este es un caso de uso muy común en comercio electrónico.

**Objetivo**: Dado un comentario de un cliente, queremos extraer automáticamente:
- El sentimiento general (positivo/negativo/neutro)
- Si el cliente está enojado
- Qué producto compró
- Qué marca es

Esto permite procesar miles de reviews automáticamente para detectar problemas, medir satisfacción, etc.

In [ ]:
# Review real de un producto (simulada pero realista)
review_producto = """
Compré estos auriculares porque tenían buenas reseñas y el precio era razonable.
La verdad que llegaron rápido, en dos días ya los tenía. El sonido es bastante bueno
para el precio, se escucha claro y los graves están bien. Lo que no me gustó es que
después de un mes de uso, el auricular derecho empezó a sonar más bajo que el izquierdo.
Contacté al vendedor y me mandaron otro par sin problema. El servicio de atención fue
excelente, respondieron al toque y resolvieron todo. En general estoy conforme, pero
esperaba que duraran más antes de tener problemas.
"""

### Construcción del Prompt

Un buen prompt es específico y claro sobre qué formato queremos en la respuesta. Acá aplicamos varias técnicas:

1. **Instrucciones claras**: Describimos exactamente qué queremos extraer
2. **Formato estructurado**: Pedimos JSON para poder parsear programáticamente
3. **Manejo de casos edge**: Indicamos qué hacer si falta información
4. **Delimitadores**: Usamos triple comillas para delimitar el texto a analizar

In [ ]:
prompt = f"""
Analiza la siguiente review de producto y extrae la información en formato JSON.

Información a extraer:
- Sentimiento: positivo, negativo o neutro
- Enojado: true o false (¿el cliente expresa enojo?)
- Producto: qué producto compró
- Problema: si hubo algún problema, cuál fue
- Calidad_atencion: cómo fue el servicio al cliente

Si alguna información no está presente, usá "no_especificado".
El campo "Enojado" debe ser booleano (true/false).

Review: ```{review_producto}```

Responde únicamente con el JSON, sin texto adicional.
"""

respuesta = get_completion(prompt)
print("ANÁLISIS DE LA REVIEW:")
print(respuesta)

### Análisis del Resultado

Observá que el modelo pudo:
1. Identificar que el sentimiento es mixto/neutro (hay aspectos positivos y negativos)
2. Detectar que no hay enojo explícito
3. Extraer el producto específico
4. Identificar el problema técnico
5. Valorar positivamente la atención al cliente

Todo esto en formato JSON que podemos usar programáticamente:

```python
import json
datos = json.loads(respuesta)
print(datos['Sentimiento'])  # Acceso a los datos
```

## Ejemplo 2: Análisis de Noticias Periodísticas

Ahora vamos a trabajar con un texto periodístico y extraer información estructurada relevante. Este tipo de análisis es útil para:
- Sistemas de monitoreo de medios
- Agregadores de noticias
- Análisis de cobertura mediática
- Detección de temas trending

In [ ]:
# Nota periodística simulada
noticia = """
BUENOS AIRES - El Ministerio de Cultura anunció ayer la apertura de tres nuevos
espacios culturales en barrios de la zona sur de la ciudad. Los centros culturales
se ubicarán en Parque Patricios, Pompeya y Nueva Pompeya, y comenzarán a funcionar
en marzo de 2024.

Según informó la ministra de Cultura, María Rodríguez, los espacios contarán con
talleres gratuitos de música, teatro, danza y artes visuales. "Es fundamental
descentralizar la oferta cultural y llegar a todos los barrios de la ciudad",
afirmó la funcionaria durante la conferencia de prensa.

La inversión total del proyecto asciende a $450 millones y se espera que beneficie
directamente a más de 15.000 vecinos de la zona sur. Las inscripciones para los
talleres comenzarán en febrero a través de la web oficial del Ministerio.
"""

In [ ]:
prompt_noticia = f"""
Analiza la siguiente noticia y extrae la información clave en formato JSON:

- Tema_principal: tema central de la noticia
- Ubicacion: dónde ocurre el hecho
- Fecha_evento: cuándo ocurrirá o ocurrió
- Protagonistas: personas u organizaciones mencionadas
- Datos_numericos: cifras relevantes (monto, cantidad de personas, etc.)
- Categoria: clasificá en política, economía, cultura, sociedad, deportes, tecnología

Si no hay información, usá "no_especificado".

Noticia: ```{noticia}```

Devuelve solo el JSON.
"""

analisis_noticia = get_completion(prompt_noticia)
print("ANÁLISIS DE LA NOTICIA:")
print(analisis_noticia)

## Ejemplo 3: Clasificación de Comentarios de Redes Sociales

Las redes sociales generan millones de comentarios por día. Los LLMs pueden ayudarnos a clasificarlos automáticamente según diferentes criterios.

**Caso de uso**: Moderación de contenido, análisis de opinión pública, detección de toxicidad.

In [ ]:
# Colección de comentarios simulados
comentarios = [
    "Excelente la nueva temporada! La mejor serie que vi en años 🔥",
    "No entiendo por qué le dan tanto hype, es bastante meh",
    "Alguien sabe dónde puedo conseguir las zapatillas que usa el prota?",
    "SPOILER: No puedo creer que mataron a ese personaje!!! 😭",
    "Qué basura, perdí 3 horas de mi vida viendo esto"
]

In [ ]:
# Procesamos múltiples comentarios
print("CLASIFICACIÓN DE COMENTARIOS:\n")

for i, comentario in enumerate(comentarios, 1):
    prompt_comentario = f"""
    Clasifica este comentario según:
    - Sentimiento: positivo, negativo o neutral
    - Tipo: opinion, pregunta, spoiler, spam, otro
    - Toxicidad: baja, media, alta
    
    Comentario: ```{comentario}```
    
    Devuelve solo un JSON con esas tres claves.
    """
    
    resultado = get_completion(prompt_comentario)
    print(f"Comentario {i}: {comentario}")
    print(f"Análisis: {resultado}")
    print("-" * 80)

## Experimentación: Probá Modificar los Parámetros

Esta sección está diseñada para que experimentes con diferentes configuraciones y veas cómo cambian los resultados.

### Ejercicio 1: Efecto de la temperatura

Ejecutá la siguiente celda varias veces cambiando el parámetro `temperature` y observá cómo varían las respuestas.

In [ ]:
# Zona de experimentación - Modifica estos valores
TEMPERATURA = 0.0  # Probá con: 0.0, 0.5, 1.0, 1.5
MODELO = "gpt-4o-mini"  # Probá con: "gpt-4o-mini", "gpt-4o"

texto_ejemplo = "El partido estuvo increíble, se definió en el último minuto."

prompt_exp = f"""
Reescribe este texto de tres formas diferentes manteniendo el significado:
{texto_ejemplo}
"""

print(f"Usando temperatura={TEMPERATURA} y modelo={MODELO}")
print("\nResultado:")
print(get_completion(prompt_exp, model=MODELO, temperature=TEMPERATURA))

### Ejercicio 2: Tus propios ejemplos

Ahora probá con tus propios textos. Puede ser:
- Un comentario de YouTube
- Una review de un producto
- Un titular de noticia
- Un tweet
- Un fragmento de conversación

Modificá las variables y experimentá con diferentes prompts.

In [ ]:
# TU TURNO: Reemplaza este texto con el que quieras analizar
mi_texto = """
Acá pega tu texto para analizar...
"""

# Modifica este prompt según lo que quieras hacer
mi_prompt = f"""
Analiza este texto y decime:
1. De qué se trata
2. Qué sentimiento expresa
3. Si hay algo interesante o relevante

Texto: ```{mi_texto}```
"""

print(get_completion(mi_prompt))

## Resumen y Conceptos Clave

### ¿Qué aprendimos?

1. **APIs de LLMs**: Interfaces que nos permiten acceder a modelos de lenguaje potentes sin necesidad de entrenarlos nosotros

2. **Configuración segura**: Cómo manejar API keys sin exponerlas en el código

3. **Función wrapper**: `get_completion()` simplifica el uso de la API

4. **Parámetros configurables**:
   - `model`: Qué modelo usar (calidad vs costo vs velocidad)
   - `temperature`: Control de creatividad/determinismo

5. **Prompting estructurado**: Técnicas para obtener respuestas en formato JSON útil

6. **Casos de uso reales**: Reviews, noticias, comentarios de redes sociales

### Técnicas de Prompting Aprendidas

- **Instrucciones claras**: Especificar exactamente qué queremos
- **Formato de salida**: Pedir JSON para procesar programáticamente
- **Manejo de casos edge**: Indicar qué hacer con información faltante
- **Delimitadores**: Usar ``` o """ para marcar el texto a procesar
- **Ejemplos específicos**: Dar formato de respuesta esperada

### Próximos pasos

En el siguiente cuaderno vamos a profundizar en:
- **System prompts**: Cómo definir el comportamiento del asistente
- **Conversaciones multi-turno**: Mantener contexto entre mensajes
- **Roles**: Diferencia entre system, user y assistant

## Glosario

**API (Application Programming Interface)**: Conjunto de definiciones y protocolos que permite que dos sistemas se comuniquen

**LLM (Large Language Model)**: Modelo de lenguaje entrenado con grandes cantidades de texto que puede generar, analizar y transformar texto

**Prompt**: Instrucción o pregunta que le damos al modelo de lenguaje

**Temperature**: Parámetro que controla la aleatoriedad de las respuestas (0=determinístico, 2=muy creativo)

**Token**: Unidad básica de procesamiento de texto (aproximadamente 0.75 palabras en español)

**JSON**: Formato de intercambio de datos estructurado, fácil de leer para humanos y máquinas

**Endpoint**: URL específica de una API donde enviamos nuestros requests

**Request**: Petición que hacemos a la API con nuestro prompt

**Response**: Respuesta que recibimos de la API después de procesar nuestro request

## Preguntas Frecuentes

**P: ¿Por qué usar temperature=0 en algunos casos?**

R: Cuando necesitamos respuestas consistentes y determinísticas, especialmente para extracción de datos estructurados. Con temperature=0, el modelo siempre va a dar la misma respuesta para el mismo prompt.

---

**P: ¿Cuánto cuesta usar la API de OpenAI?**

R: Depende del modelo. GPT-4o-mini es el más económico (~$0.15 por millón de tokens de entrada). Para este curso, $5 USD es más que suficiente.

---

**P: ¿Qué pasa si mi API key se vence o me quedo sin crédito?**

R: La API va a devolver un error. Podes cargar más crédito en tu cuenta de OpenAI o esperar a que se renueve si tenés un plan de subscripción.

---

**P: ¿Puedo usar estos modelos sin conexión a internet?**

R: No, la API de OpenAI requiere conexión. Sin embargo, en clases posteriores vamos a ver Ollama, que permite ejecutar modelos localmente.

---

**P: ¿El modelo tiene acceso a internet o a información actualizada?**

R: No, el modelo solo tiene conocimiento hasta su fecha de entrenamiento (típicamente varios meses atrás). En el último cuaderno del módulo veremos cómo darle acceso a información actualizada mediante búsqueda web.

## Referencias y Recursos Adicionales

**Documentación oficial**:
- OpenAI API Documentation: https://platform.openai.com/docs
- Guía de modelos: https://platform.openai.com/docs/models
- Pricing: https://openai.com/pricing

**Recursos recomendados**:
- Curso de DeepLearning.AI sobre Prompt Engineering (gratuito)
- OpenAI Cookbook: Ejemplos prácticos en GitHub
- Comunidad de OpenAI en Discord

**Papers relevantes**:
- "Language Models are Few-Shot Learners" (GPT-3)
- "Training language models to follow instructions" (InstructGPT)